In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Flatten, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests,os
from datetime import datetime, timedelta

def fetch_news_and_sentiment(ticker, start_date, end_date):
    total_responses=1000
    headlines = []
    dates = []
    curr_page=1
    while len(headlines)<total_responses-5:
        api_url = f"https://newsapi.org/v2/everything?q={ticker}&from={start_date}&to={end_date}&apiKey=7e9676fa277d423f8deff9a4894bd717&page={curr_page}&language=en"
        response = requests.get(api_url)
        news_data = response.json()
        total_responses=news_data['totalResults']
        #print(f'news_data: {news_data}\n\n')
    

        #print(news_data['totalResults'])
        #print(len(news_data['articles']))
        if "articles" in news_data:
            for article in news_data['articles']:
                headlines.append(article['title'])
                # Extract the publication date of the article
                dates.append(article['publishedAt'][:10])  # Extract the YYYY-MM-DD format
        #print(dates)
        curr_page+=1

    print(f"length of headlines: {len(headlines)}")


    
    # Perform sentiment analysis using VADER
    analyzer = SentimentIntensityAnalyzer()
    sentiment_data = []
    for headline, date in zip(headlines, dates):
        score = analyzer.polarity_scores(headline)
        sentiment_data.append({'date': date, 'sentiment': score['compound']})  # Store date and compound score
        #print(headline)
        #print(score['compound']) 
    return sentiment_data
#start_date = "2024-10-30"
#end_date = "2024-11-28"
#sentiment_data=fetch_news_and_sentiment('TSLA', start_date, end_date)

def aggregate_daily_sentiment(sentiment_data, start_date, end_date):
    # Convert to DataFrame
    sentiment_df = pd.DataFrame(sentiment_data)
    # print(sentiment_df[:30])
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])

    # Group by date and calculate the average sentiment for each day
    daily_sentiment = (
        sentiment_df.groupby('date')['sentiment']
        .mean()
        .reindex(pd.date_range(start_date, end_date), fill_value=0)  # Fill missing days with 0 sentiment
    )
    return daily_sentiment


start_date = "2024-11-02"
end_date = "2024-12-01"


sentiment_data=fetch_news_and_sentiment("TSLA", start_date, end_date)
print(sentiment_data)
daily_sentiment=aggregate_daily_sentiment(sentiment_data, start_date, end_date)
print(daily_sentiment)


zero_sentiment_dates = daily_sentiment[daily_sentiment == 0.0].index
print(zero_sentiment_dates)
for date in zero_sentiment_dates:
    for entry in sentiment_data:
        if entry['date']==date:
            print(f"{date}: {entry['sentiment']}")
            #This just checks if we are putting 0.0 for dates that have articles. Ideally, there should be no output
            


length of headlines: 294
[{'date': '2024-11-06', 'sentiment': 0.0}, {'date': '2024-11-06', 'sentiment': 0.4767}, {'date': '2024-11-06', 'sentiment': 0.5267}, {'date': '2024-11-25', 'sentiment': 0.0}, {'date': '2024-11-06', 'sentiment': 0.4588}, {'date': '2024-11-08', 'sentiment': 0.5859}, {'date': '2024-11-11', 'sentiment': 0.0}, {'date': '2024-11-18', 'sentiment': 0.7906}, {'date': '2024-11-27', 'sentiment': -0.4019}, {'date': '2024-11-16', 'sentiment': 0.4754}, {'date': '2024-11-02', 'sentiment': 0.0}, {'date': '2024-11-13', 'sentiment': 0.4404}, {'date': '2024-11-26', 'sentiment': -0.5574}, {'date': '2024-11-11', 'sentiment': 0.0}, {'date': '2024-11-11', 'sentiment': 0.6597}, {'date': '2024-11-13', 'sentiment': 0.2263}, {'date': '2024-11-25', 'sentiment': -0.5423}, {'date': '2024-11-04', 'sentiment': 0.3376}, {'date': '2024-11-11', 'sentiment': 0.0}, {'date': '2024-11-13', 'sentiment': 0.3182}, {'date': '2024-11-16', 'sentiment': 0.6249}, {'date': '2024-11-07', 'sentiment': 0.0}, {'